# Plotly - Enteric Fermentation

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px  
import plotly.graph_objects as go
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output

## Loading Datasets

- **Enteric Fermentation**

In [2]:
ef = pd.read_csv("datasets/enteric_fermentation_countries.csv")

In [3]:
ef.head()

,Domain,Area,Element,Item,Year,Source,Unit,Value
0,Enteric Fermentation,Brazil,Stocks,"Cattle, dairy",1961,FAO TIER 1,Head,7.396200e+06
1,Enteric Fermentation,Brazil,Emissions (CH4),"Cattle, dairy",1961,FAO TIER 1,kilotonnes,5.325264e+02
2,Enteric Fermentation,Brazil,Stocks,"Cattle, dairy",1962,FAO TIER 1,Head,7.617600e+06
3,Enteric Fermentation,Brazil,Emissions (CH4),"Cattle, dairy",1962,FAO TIER 1,kilotonnes,5.484672e+02
4,Enteric Fermentation,Brazil,Stocks,"Cattle, dairy",1963,FAO TIER 1,Head,7.907600e+06


# Data Preparation

In [4]:
ef.drop(["Domain","Source"],axis =1,inplace=True)

In [5]:
for index in ef.index:
    if ef.loc[index,'Element']=='Stocks':
        ef.loc[index,'Heads'] =  ef.loc[index,'Value']
    if ef.loc[index,'Element']=='Emissions (CH4)':
        ef.loc[index,'Enteric CH4 Emissions (kt)'] =  ef.loc[index,'Value']

In [6]:
ef = ef.drop(columns = ["Element","Unit","Value"])

In [7]:
ef = ef.groupby(['Item','Year',"Area"], as_index = False).sum()

In [8]:
ef = ef.rename(columns={"Item":"Animal","Area":"Country"})
ef.head(10)

,Animal,Year,Country,Heads,Enteric CH4 Emissions (kt)
0,"Cattle, dairy",1961,Brazil,7396200.0,532.5264
1,"Cattle, dairy",1961,China,506557.0,34.4459
2,"Cattle, dairy",1961,Ireland,1206000.0,141.1020
3,"Cattle, dairy",1961,United States of America,17243008.0,2207.1050
4,"Cattle, dairy",1962,Brazil,7617600.0,548.4672
5,"Cattle, dairy",1962,China,526918.0,35.8304
6,"Cattle, dairy",1962,Ireland,1220000.0,142.7400
7,"Cattle, dairy",1962,United States of America,16842000.0,2155.7760
8,"Cattle, dairy",1963,Brazil,7907600.0,569.3472
9,"Cattle, dairy",1963,China,537816.0,36.5715


In [9]:
all_animals = ef.Animal.unique()
all_animals


array(['Cattle, dairy', 'Cattle, non-dairy'], dtype=object)

In [10]:
all_countries = ef.Country.unique()
all_countries

array(['Brazil', 'China', 'Ireland', 'United States of America'],
      dtype=object)

## Creating the Dash Components

In [11]:
app = dash.Dash(__name__)

In [12]:
app.layout = html.Div([
    
    html.H1("Methane Emissions from the Enteric Fermentation", style={"color":"white","backgroundColor":"green","text-align": "center"}),
    html.H2("Irish AgriBusiness and Top Countries Producers", style={"color":"white","backgroundColor":"green","text-align": "center"}),
    html.Br(),
    html.Br(),
    
    dcc.Checklist(
        id="animal_checklist",
        options=[{"label":x,"value":x}
                 for x in all_animals],
        labelStyle={"display":"inline=block"}
    ), 
     html.Br(),
    
    dcc.Checklist(
        id="country_checklist",
        options=[{"label":x,"value":x}
                 for x in all_countries],
        labelStyle={"display":"inline=block"}
    ), 
    
    dcc.Graph(id="line-chart",figure={}),
])          

In [13]:
# Connect the Plotly graphs with Dash Components
@app.callback(
    Output("line-chart","figure"),
    [Input(component_id="animal_checklist",component_property="value"),
     Input(component_id="country_checklist",component_property="value")])
    
def update_line_chart(animal_selected,country_selected):
    eff=ef[ef["Animal"== animal_selected]],
    eff = eff[eff["Country"]==country_selected]
    fig = px.line(eff,x="Year",y="Enteric CH4 Emissions (kt)",color="Animal")
    fig.update_traces(mode="markers+lines",hovertemplate=None)
    fig.update_layout(hovermode="x unified")
    return fig

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Nov/2021 22:57:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2021 22:57:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2021 22:57:02] "GET /_dash-dependencies HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py", line 2895, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1675, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1683, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: False

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle

127.0.0.1 - - [29/Nov/2021 22:57:02] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py", line 2895, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1675, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1683, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: False

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle

...

127.0.0.1 - - [29/Nov/2021 23:20:04] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py", line 2895, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1675, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1683, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: False

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle

127.0.0.1 - - [29/Nov/2021 23:20:04] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py", line 2895, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1675, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1683, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: False

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle

127.0.0.1 - - [29/Nov/2021 23:20:05] "POST /_dash-update-component HTTP/1.1" 500 -
